Cargar los archivos CSV

In [ ]:
import pandas as pd

# Cargar los archivos CSV
calificaciones_df = pd.read_csv('input/calificaciones.csv')
carrera_df = pd.read_csv('input/carrera.csv')
escuelas_df = pd.read_csv('input/escuelas.csv')
estudiantes_df = pd.read_csv('input/estudiantes.csv')

# Ver las primeras filas de cada DataFrame
print(calificaciones_df.head())
print(carrera_df.head())
print(escuelas_df.head())
print(estudiantes_df.head())

Explorar los datos
Verificamos si los datos contienen valores nulos, duplicados, o alguna inconsistencia:

In [ ]:
# Verificar si hay valores nulos
print("Valores nulos en calificaciones:\n", calificaciones_df.isnull().sum())
print("Valores nulos en carrera:\n", carrera_df.isnull().sum())
print("Valores nulos en escuelas:\n", escuelas_df.isnull().sum())
print("Valores nulos en estudiantes:\n", estudiantes_df.isnull().sum())

# Verificar si hay duplicados
print("Duplicados en calificaciones:", calificaciones_df.duplicated().sum())
print("Duplicados en carrera:", carrera_df.duplicated().sum())
print("Duplicados en escuelas:", escuelas_df.duplicated().sum())
print("Duplicados en estudiantes:", estudiantes_df.duplicated().sum())

Limpieza de los datos
Dependiendo de lo que encuentres en la exploración, puedes hacer lo siguiente:

Eliminar duplicados:

In [ ]:
# Eliminar duplicados
calificaciones_df = calificaciones_df.drop_duplicates()
carrera_df = carrera_df.drop_duplicates()
escuelas_df = escuelas_df.drop_duplicates()
estudiantes_df = estudiantes_df.drop_duplicates()

Manejo de valores nulos:

Dependiendo de los valores nulos, puedes optar por eliminar filas o rellenarlas:

In [ ]:
# Rellenar valores nulos con un valor específico (ejemplo: 0 o 'Desconocido')
calificaciones_df = calificaciones_df.fillna({'calificacion': 0, 'tipo_calificacion': 'Desconocido'})
carrera_df = carrera_df.fillna({'nombre': 'Desconocido', 'corto': 'Desconocido'})
escuelas_df = escuelas_df.fillna({'escuela': 'Desconocida'})
estudiantes_df = estudiantes_df.fillna({'prepa': 'Desconocida', 'colonia': 'Desconocida', 'municipio': 'Desconocido'})

# O eliminar filas con valores nulos si es necesario
# calificaciones_df = calificaciones_df.dropna()


Reemplazar valores incorrectos o inconsistentes:

Si encuentras valores que no son correctos (por ejemplo, valores numéricos negativos donde no deberían ser), puedes reemplazarlos:

In [ ]:
# Reemplazar valores negativos por 0 en calificación
calificaciones_df['calificacion'] = calificaciones_df['calificacion'].apply(lambda x: max(x, 0))


Revisar tipos de datos
Es importante verificar si los tipos de datos son correctos (por ejemplo, fechas como strings, números como strings, etc.).

In [ ]:
# Verificar tipos de datos
print(estudiantes_df.dtypes)

# Convertir columnas a tipos correctos si es necesario
estudiantes_df['ingreso'] = pd.to_datetime(estudiantes_df['ingreso'], errors='coerce')
estudiantes_df['termino'] = pd.to_datetime(estudiantes_df['termino'], errors='coerce')


Unir los DataFrames
Si tienes que combinar los DataFrames en uno solo para hacer el análisis, usa merge():

In [ ]:
# Verificar las columnas de los DataFrames
print(estudiantes_df.columns)
print(escuelas_df.columns)

# Unir estudiantes con carreras
estudiantes_calificaciones_escuelas = pd.merge(estudiantes_df, calificaciones_df, on='clave_alumno', how='left')
estudiantes_carrera = pd.merge(estudiantes_calificaciones_escuelas, carrera_df, on='clave_carrera', how='left')

# Unir con escuelas usando 'clave_carrera' de estudiantes y 'clave' de escuelas
estudiantes_completos = pd.merge(estudiantes_carrera, escuelas_df, left_on='clave_carrera', right_on='clave', how='left')

# Verificar el resultado
print(estudiantes_completos.head())


Errores ortográficos en claves y datos categóricos:
- Normalizar cadenas:
- Reemplazar errores comunes:

In [ ]:
estudiantes_df['ciudad'] = estudiantes_df['ciudad'].str.lower().str.strip()
escuelas_df['escuela'] = escuelas_df['escuela'].str.lower().str.strip()

estudiantes_df['ciudad'] = estudiantes_df['ciudad'].replace({
    'cdmx': 'ciudad de mexico', 
    'mexico': 'ciudad de mexico'
})


Formato de fechas incorrecto:

In [ ]:
estudiantes_df['ingreso'] = pd.to_datetime(estudiantes_df['ingreso'], errors='coerce')
estudiantes_df['termino'] = pd.to_datetime(estudiantes_df['termino'], errors='coerce')


Números con letras (mezcla de datos numéricos y texto):

In [ ]:
estudiantes_df['ingreso'] = pd.to_numeric(estudiantes_df['ingreso'], errors='coerce')
estudiantes_df['termino'] = pd.to_numeric(estudiantes_df['termino'], errors='coerce')

Detectar filas con datos incorrectos:

In [ ]:
# Convertir las columnas a numérico, reemplazando los valores no numéricos por NaN
estudiantes_df['ingreso'] = pd.to_numeric(estudiantes_df['ingreso'], errors='coerce')
estudiantes_df['termino'] = pd.to_numeric(estudiantes_df['termino'], errors='coerce')


Números con letras (mezcla de datos numéricos y texto):

In [ ]:

# Filtrar las filas donde 'ingreso' o 'termino' es NaN (lo que indica que eran datos no numéricos)
ingreso_no_numerico = estudiantes_df[estudiantes_df['ingreso'].isna()]
termino_no_numerico = estudiantes_df[estudiantes_df['termino'].isna()]

# Mostrar las filas con datos no numéricos en 'ingreso' o 'termino'
print("Filas con ingreso no numérico:")
print(ingreso_no_numerico)
print("\nFilas con termino no numérico:")
print(termino_no_numerico)


Reemplazo de valores nulos después de la limpieza:

In [ ]:
estudiantes_completos['ciudad'].fillna('Desconocida', inplace=True)
estudiantes_completos['ingreso'].fillna(0, inplace=True)
estudiantes_completos['termino'].fillna(0, inplace=True)
estudiantes_completos['escuela'].fillna('Desconocida', inplace=True)


Eliminar filas con valores nulos en columnas clave (calificación, materia, etc.)

In [ ]:
# Eliminar filas con valores nulos en las columnas de calificaciones
estudiantes_completos_limpios = estudiantes_completos.dropna(subset=['periodo', 'clave_materia', 'calificacion', 'tipo_calificacion'])

# Guardar el DataFrame limpio
estudiantes_completos_limpios.to_csv('estudiantes_completos_limpios.csv', index=False)


Rellenar valores nulos con un valor predeterminado (por ejemplo, 0 para las calificaciones)

In [ ]:
# Rellenar los valores nulos en las columnas clave con 0 o algún valor predeterminado
estudiantes_completos['calificacion'] = estudiantes_completos['calificacion'].fillna(0)
estudiantes_completos['periodo'] = estudiantes_completos['periodo'].fillna('Desconocido')  # O algún valor que se adapte
estudiantes_completos['clave_materia'] = estudiantes_completos['clave_materia'].fillna('Desconocido')
estudiantes_completos['tipo_calificacion'] = estudiantes_completos['tipo_calificacion'].fillna('Desconocido')
estudiantes_completos['clave'] = estudiantes_completos['clave'].fillna('Desconocido')  # Para la escuela

# Guardar el DataFrame limpio
estudiantes_completos.to_csv('estudiantes_completos_limpios.csv', index=False)


Solo verificar y visualizar las filas con valores nulos

In [ ]:
# Ver las filas con valores nulos en las columnas clave
nulos = estudiantes_completos[estudiantes_completos.isnull().any(axis=1)]
print(nulos)

In [ ]:
# Convertir todas las columnas numéricas a tipo float
estudiantes_df['ingreso'] = pd.to_numeric(estudiantes_df['ingreso'], errors='coerce', downcast='float')
estudiantes_df['termino'] = pd.to_numeric(estudiantes_df['termino'], errors='coerce', downcast='float')

# También se puede aplicar a otras columnas numéricas si las hubiera
# Asegurarse de que todas las columnas numéricas sean float
for col in estudiantes_df.select_dtypes(include=['int64', 'float64']).columns:
    estudiantes_df[col] = estudiantes_df[col].astype('float')

# Verificar el tipo de datos después de la conversión
print(estudiantes_df.dtypes)

Verificación final y exportación
Después de la limpieza, puedes revisar el DataFrame final y guardarlo para su análisis posterior:

In [ ]:
# Verificar si hay valores nulos en el DataFrame final
print(estudiantes_completos.isnull().sum())

# Guardar el DataFrame limpio
estudiantes_completos.to_csv('estudiantes_completos_limpios.csv', index=False)

In [ ]:
# Cargar el archivo CSV para visualizarlo
estudiantes_completos_guardados = pd.read_csv('estudiantes_completos_limpios.csv')

# Ver las primeras filas
print(estudiantes_completos_guardados.head())
